# 1. Experimental frame setup

## 1.1 Dataset download
1. UrbanSound8K
Download the dataset from https://serv.cusp.nyu.edu/projects/urbansounddataset/download-urbansound8k.html .

2. Extracted Features - long60 
Download the extracted features from 

3. Extracted Features - overlap7525
Download the extracted features from 

## 1.2 Feature extraction
Starting from the .wav files of the UrbanSound8K dataset, we use the $\texttt{librosa}$ library to obtain a numerical representation. We save the extracted features and labels in two $\texttt{.npy}$ files that we can easily load in memory. This is crucial in the process of cross-validation, where we would otherwise lose a lot of time reloading the files with $\texttt{librosa}$.

In [8]:
from preprocessor import preprocessor

pp = preprocessor()

train_dirs = ["fold1", "fold2", "fold3", "fold4", "fold5", "fold6", "fold7", "fold8", "fold9", "fold10"]

#pp.save_fts_lbs(train_dirs=train_dirs, save_path='extracted_short_60', segment_size=20480, overlap=0.5, bands=60, frames=41)
#pp.save_fts_lbs(train_dirs=train_dirs, save_path='extracted_long_60', segment_size=51200, overlap=0.9, bands=200, frames=101)
#pp.save_fts_lbs(train_dirs=train_dirs, save_path='extracted_short_200', segment_size=20480, overlap=0.5, bands=200, frames=41)
#pp.save_fts_lbs(train_dirs=train_dirs, save_path='extracted_long_200', segment_size=51200, overlap=0.9, bands=200, frames=101)

## 1.3 Overlapping of segments

# 2. Piczak CNN
## 2.1 Cross Validation
For each model (i.e. short60, long60, short200, long200) we ran 10-fold cross-validation, using eight folders as training data, one folder as validation set and one folder as test set. 

In [ ]:
from train_models import piczac_cross_validation

piczac_cross_validation(epochs=300, load_path='extracted_short_60')
piczac_cross_validation(epochs=300, load_path='extracted_long_60')
piczac_cross_validation(epochs=300, load_path='extracted_short_200')
piczac_cross_validation(epochs=300, load_path='extracted_long_200')

If you want to see the training process, you can download the TensorBoard folder from . 

Below you can see the final results.

In [4]:
import pandas as pd

df = pd.read_excel('logs/piczak_cv_results.xlsx', sheet_name=None)
df.head()

,run1,run2,run3,run4,run5,run6,run7,run8,run9,run10,Average,StDev
short_60,0.631746,0.524219,0.519109,0.614269,0.623094,0.550229,0.555519,0.587509,0.647792,0.675970,0.583721,0.048142
long_60,0.690345,0.671870,0.518753,0.651414,0.683282,0.515301,0.623859,0.548439,0.703675,0.737907,0.634484,0.080128
short_200,0.568919,0.584927,0.511141,0.590961,0.631396,0.576656,0.583974,0.660578,0.658915,0.646573,0.601404,0.047432
long_200,0.639606,0.586811,0.563517,0.670743,0.634963,0.564297,0.609190,0.704243,0.675298,0.601156,0.624982,0.048254


## 2.1 Confusion Matrix

In [8]:
import train_models as tm

model_filename = '/Users/lucacambiaghi/OneDrive - Danmarks Tekniske Universitet/Deep Learning Data/models/long60/long60_150_(1, 2).h5'
load_path = '/Users/lucacambiaghi/OneDrive - Danmarks Tekniske Universitet/Deep Learning Data/extracted/long_60'

tm.plot_confusion_matrix(model_filename, load_path)

Confusion matrix, without normalization
[[627   0   8  18   1  28   0 211   0   0]
 [  0 107   6   0  34   0   7   0   0  28]
 [ 43   0 771   3   0   0   0   0  14  59]
 [  0   0  56 536   1   2  18   0   0   0]
 [ 17   0   4  15 511   0  22  96   0   2]
 [297   0  76   2   0 433   0  64   4  14]
 [  0   0   3   7   0   0  47   1   0   9]
 [157   0   0   0  39 659   3  69   0   6]
 [ 48   0  88   0   0  16   0   0 642  25]
 [ 55   0 155   0  26   0   0   0   1 663]]


## 2.2 Boxplot
https://plot.ly/matplotlib/box-plots/#multiple-box-plots

# 3. Piczak cnn on overlapping sounds